# SW-5-LinkedData

**Navigation** : [<< 4-SPARQL](SW-4-SPARQL.ipynb) | [Index](README.md) | [6-RDFS >>](SW-6-RDFS.ipynb)

## Donnees Liees : DBpedia, Wikidata et le Web de donnees

Ce notebook explore les principes du Linked Data et leur mise en pratique a travers l'interrogation de deux bases de connaissances majeures du Web semantique : DBpedia et Wikidata. Nous apprendrons a combiner ces sources avec des requetes federees.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Expliquer les principes du Linked Data
2. Interroger DBpedia et Wikidata avec SPARQL
3. Combiner des sources avec les requetes federees
4. Gerer les erreurs et timeouts des endpoints distants

### Prerequis
- Notebook SW-4-SPARQL complete
- Connexion Internet requise

### Duree estimee : 50 minutes

## Installation et imports

Nous utilisons **dotNetRDF 3.2.1**. La classe `SparqlRemoteEndpoint` permet d'interroger des endpoints SPARQL distants comme DBpedia ou Wikidata.

In [ ]:
#r "nuget: dotNetRDF, 3.2.1"

Installing Packages dotNetRDF

In [ ]:
using System;
using System.Linq;
using System.Collections.Generic;
using VDS.RDF;
using VDS.RDF.Query;
using VDS.RDF.Writing;
using VDS.RDF.Parsing;

Console.WriteLine("dotNetRDF charge avec succes.");

dotNetRDF charge avec succes.


---

## 1. Principes du Linked Data

Le **Linked Data** (donnees liees) est une methode de publication de donnees structurees sur le Web, permettant de les interconnecter. Il constitue le socle du **Web Semantique**.

### Les 4 regles de Tim Berners-Lee (2006)

| # | Regle | Description | Exemple |
|---|-------|-------------|--------|
| 1 | **Utiliser des URIs** | Identifier chaque chose par un URI | `http://dbpedia.org/resource/Paris` |
| 2 | **Utiliser des URIs HTTP** | Permettre la resolution via le Web | Un navigateur peut acceder a l'URI |
| 3 | **Fournir des informations utiles** | Retourner des donnees RDF quand on accede a un URI | Content negotiation : HTML ou RDF |
| 4 | **Inclure des liens** | Lier vers d'autres URIs pour permettre la navigation | DBpedia lie vers Wikidata, GeoNames |

### Le schema 5 etoiles de l'Open Data

| Etoiles | Exigences | Exemple |
|---------|-----------|--------|
| * | Donnees sur le Web, licence ouverte | PDF scan |
| ** | Donnees structurees lisibles par machine | Fichier Excel |
| *** | Format non-proprietaire | CSV au lieu d'Excel |
| **** | URIs pour identifier les choses (RDF) | RDF avec URIs |
| ***** | **Linked Data** : liens vers d'autres sources | DBpedia liant vers Wikidata |

### Architecture

```
  Application / Navigateur
           |
     Requete HTTP
           |
   +-------v--------+     +------------------+
   | Endpoint SPARQL |<--->| Triplestore RDF  |
   | (DBpedia, etc.) |     | (milliards de    |
   +----------------+     |  triplets)        |
           |               +------------------+
     Reponse SPARQL
     (XML, JSON, CSV)
```

> **Le Linked Open Data Cloud** : Des milliers de datasets interconnectes forment un graphe mondial de connaissances. DBpedia et Wikidata en sont le coeur.

---

## 2. DBpedia : Interrogation via SPARQL

**DBpedia** extrait et structure le contenu de Wikipedia en RDF. Son endpoint SPARQL est `http://dbpedia.org/sparql`.

| Prefixe | URI | Contenu |
|---------|-----|--------|
| `dbr:` | `http://dbpedia.org/resource/` | Ressources (entites) |
| `dbo:` | `http://dbpedia.org/ontology/` | Ontologie (classes, proprietes) |
| `dbp:` | `http://dbpedia.org/property/` | Proprietes brutes (infobox) |

### Fonctions utilitaires

Nous definissons d'abord des fonctions pour executer des requetes et afficher les resultats avec gestion d'erreurs.

> **Important** : Les endpoints publics peuvent etre lents ou temporairement indisponibles. Toujours utiliser un `try-catch`.

In [ ]:
// Fonctions utilitaires pour les requetes SPARQL distantes

public static void ExecuteAndDisplaySparqlQuery(
    SparqlRemoteEndpoint endpoint, string query, int resultLimit = 10)
{
    try
    {
        SparqlResultSet results = endpoint.QueryWithResultSet(query);
        if (results != null && results.Count > 0)
        {
            int count = 0;
            foreach (var result in results)
            {
                Console.WriteLine(result.ToString());
                count++;
                if (count >= resultLimit)
                {
                    Console.WriteLine($"...affiche {resultLimit} resultats sur {results.Count}.");
                    break;
                }
            }
        }
        else
        {
            Console.WriteLine("Aucun resultat trouve.");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Erreur SPARQL : {ex.Message}");
    }
}

public static void ExecuteAndDisplaySparqlDescribe(
    SparqlRemoteEndpoint endpoint, string query, int resultLimit = 10)
{
    try
    {
        IGraph graph = endpoint.QueryWithResultGraph(query);
        if (graph != null && graph.Triples.Count > 0)
        {
            int count = 0;
            foreach (var triple in graph.Triples)
            {
                Console.WriteLine(triple.ToString());
                count++;
                if (count >= resultLimit)
                {
                    Console.WriteLine($"...affiche {resultLimit} triples sur {graph.Triples.Count}.");
                    break;
                }
            }
        }
        else
        {
            Console.WriteLine("Aucun triple trouve.");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Erreur SPARQL DESCRIBE : {ex.Message}");
    }
}

Console.WriteLine("Fonctions utilitaires definies.");

Fonctions utilitaires definies.



(4,5): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'

(35,5): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'



### Connexion a DBpedia et test

La classe `SparqlRemoteEndpoint` encapsule l'URI de l'endpoint et le graphe par defaut.

In [ ]:
// Connexion a l'endpoint DBpedia
SparqlRemoteEndpoint endpoint = new SparqlRemoteEndpoint(
    new Uri("http://dbpedia.org/sparql"), 
    "http://dbpedia.org"
);

// Test : lister les types (classes) disponibles
string testQuery = "SELECT DISTINCT ?Concept WHERE {[] a ?Concept} LIMIT 10";
Console.WriteLine("Test de connexion - Types dans DBpedia :");
ExecuteAndDisplaySparqlQuery(endpoint, testQuery);

Test de connexion - Types dans DBpedia :


?Concept = http://www.w3.org/2002/07/owl#FunctionalProperty


?Concept = http://www.w3.org/1999/02/22-rdf-syntax-ns#Property


?Concept = http://www.w3.org/2002/07/owl#Thing


?Concept = http://www.w3.org/2002/07/owl#Class


...

?Concept = http://xmlns.com/foaf/0.1/Organization


?Concept = http://xmlns.com/foaf/0.1/Person


?Concept = http://dbpedia.org/ontology/Company


...affiche 10 resultats sur 10.



(2,1): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'

(2,37): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'



### Requetes sur des entites DBpedia

Executons des requetes progressives sur differents types d'entites : personnes, films, livres, villes.

In [ ]:
// Requete 1 : Relations d'Albert Einstein
// Pattern : <URI_fixe> ?predicat ?objet
Console.WriteLine("=== Relations d'Albert Einstein ===");
string q1 = "SELECT ?link ?person WHERE { <http://dbpedia.org/resource/Albert_Einstein> ?link ?person . } LIMIT 15";
ExecuteAndDisplaySparqlQuery(endpoint, q1, 15);

=== Relations d'Albert Einstein ===


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://www.w3.org/2002/07/owl#Thing


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://xmlns.com/foaf/0.1/Person


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://dbpedia.org/ontology/Person


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://dbpedia.org/ontology/Person


...

?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://dbpedia.org/ontology/Animal


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://dbpedia.org/ontology/Eukaryote


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://dbpedia.org/ontology/Eukaryote


?link = http://www.w3.org/1999/02/22-rdf-syntax-ns#type , ?person = http://dbpedia.org/ontology/Scientist


...affiche 15 resultats sur 15.


In [ ]:
// Requete 2 : Films de Brad Pitt
// Pattern inverse : ?sujet dbo:starring <URI_fixe>
Console.WriteLine("=== Films de Brad Pitt ===");
string q2 = "SELECT ?film WHERE { ?film dbo:starring <http://dbpedia.org/resource/Brad_Pitt> . }";
ExecuteAndDisplaySparqlQuery(endpoint, q2);

Console.WriteLine();

// Requete 3 : Livres de J.K. Rowling
Console.WriteLine("=== Livres de J.K. Rowling ===");
string q3 = "SELECT ?book WHERE { ?book dbo:author <http://dbpedia.org/resource/J._K._Rowling> . }";
ExecuteAndDisplaySparqlQuery(endpoint, q3);

=== Films de Brad Pitt ===


?film = http://dbpedia.org/resource/Megamind


?film = http://dbpedia.org/resource/Ocean's_Thirteen


?film = http://dbpedia.org/resource/The_Assassination_of_Jesse_James_by_the_Coward_Robert_Ford


?film = http://dbpedia.org/resource/Too_Young_to_Die%3F


...

?book = http://dbpedia.org/resource/Harry_Potter_and_the_Chamber_of_Secrets


?book = http://dbpedia.org/resource/Harry_Potter_and_the_Deathly_Hallows


?book = http://dbpedia.org/resource/Harry_Potter_and_the_Goblet_of_Fire


?book = http://dbpedia.org/resource/Harry_Potter_and_the_Half-Blood_Prince


...affiche 10 resultats sur 19.


In [ ]:
// Requete 4 : Villes de France
Console.WriteLine("=== Villes de France ===");
string q4 = "SELECT ?city WHERE { ?city dbo:country <http://dbpedia.org/resource/France> . } LIMIT 15";
ExecuteAndDisplaySparqlQuery(endpoint, q4, 15);

Console.WriteLine();

// Requete 5 : Navigation multi-noeuds - Equipes Premier League et villes
// Chaine : team -> ground -> location
Console.WriteLine("=== Equipes Premier League et leurs villes ===");
string q5 = @"SELECT ?team ?city WHERE { 
    ?team dbo:league <http://dbpedia.org/resource/Premier_League> . 
    ?team dbo:ground ?ground . 
    ?ground dbo:location ?city . 
}";
ExecuteAndDisplaySparqlQuery(endpoint, q5);

=== Villes de France ===


?city = http://dbpedia.org/resource/183rd_Infantry_Division_of_Africa


?city = http://dbpedia.org/resource/Aur%C3%A9lien_Jeanney


?city = http://dbpedia.org/resource/Cat's_Eyes_(TV_series)


?city = http://dbpedia.org/resource/Cl%C3%A9ment_Chidekh


...

?team = http://dbpedia.org/resource/1994%E2%80%9395_Coventry_City_F.C._season , ?city = http://dbpedia.org/resource/England


?team = http://dbpedia.org/resource/1994%E2%80%9395_Coventry_City_F.C._season , ?city = http://dbpedia.org/resource/Coventry


?team = http://dbpedia.org/resource/1994%E2%80%9395_Coventry_City_F.C._season , ?city = http://dbpedia.org/resource/Hillfields


?team = http://dbpedia.org/resource/1995%E2%80%9396_Coventry_City_F.C._season , ?city = http://dbpedia.org/resource/England


...affiche 10 resultats sur 1250.


#### Interpretation : Requetes DBpedia

| Requete | Pattern | Concept SPARQL |
|---------|---------|----------------|
| Einstein | `<URI> ?link ?person` | Exploration de toutes les relations d'une ressource |
| Brad Pitt | `?film dbo:starring <URI>` | Selection inversee (trouver les sujets) |
| Rowling | `?book dbo:author <URI>` | Meme pattern, predicat different |
| Villes France | `?city dbo:country <URI>` | Relation geographique |
| Premier League | Chaine de 3 triplets | Navigation de graphe multi-noeuds |

> **URIs DBpedia** : Suivent le pattern `http://dbpedia.org/resource/Nom_Article_Wikipedia`. Le predicat `dbo:` vient de l'ontologie formelle, `dbp:` des infobox brutes.

### Requetes avancees : prefixes, filtres, agregation

Passons a des requetes utilisant des prefixes explicites, `ORDER BY`, `FILTER`, `OPTIONAL` et `GROUP_CONCAT`.

In [ ]:
// Requete 6 : Laureats du Nobel de physique, tries par date de naissance
string q6 = @"
PREFIX dbpedia: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?scientist ?birth 
WHERE {
  ?scientist a dbo:Scientist .
  ?scientist dbo:award dbpedia:Nobel_Prize_in_Physics .
  ?scientist dbo:birthDate ?birth
} 
ORDER BY ?birth
LIMIT 10";

Console.WriteLine("=== Laureats Nobel de physique ===");
ExecuteAndDisplaySparqlQuery(endpoint, q6);

=== Laureats Nobel de physique ===


?scientist = http://dbpedia.org/resource/Johannes_Diderik_van_der_Waals , ?birth = 1837-11-23^^http://www.w3.org/2001/XMLSchema#date


?scientist = http://dbpedia.org/resource/Johannes_Diderik_van_der_Waals , ?birth = 1837-11-23^^http://www.w3.org/2001/XMLSchema#date


?scientist = http://dbpedia.org/resource/Johannes_Diderik_van_der_Waals , ?birth = 1837-11-23^^http://www.w3.org/2001/XMLSchema#date


?scientist = http://dbpedia.org/resource/Wilhelm_R%C3%B6ntgen , ?birth = 1845-03-27^^http://www.w3.org/2001/XMLSchema#date


...

?scientist = http://dbpedia.org/resource/Gabriel_Lippmann , ?birth = 1845-08-16^^http://www.w3.org/2001/XMLSchema#date


?scientist = http://dbpedia.org/resource/Gabriel_Lippmann , ?birth = 1845-08-16^^http://www.w3.org/2001/XMLSchema#date


?scientist = http://dbpedia.org/resource/Gabriel_Lippmann , ?birth = 1845-08-16^^http://www.w3.org/2001/XMLSchema#date


?scientist = http://dbpedia.org/resource/Henri_Becquerel , ?birth = 1852-12-15^^http://www.w3.org/2001/XMLSchema#date


...affiche 10 resultats sur 10.


In [ ]:
// Requete 7 : Films d'aventure avec metadonnees enrichies
string q7 = @"
SELECT DISTINCT ?film ?number ?abstract ?name
WHERE {
   ?film dbo:wikiPageWikiLink dbr:Adventure_film .
   ?film dbo:wikiPageID ?number .
   ?film rdfs:comment ?abstract .
   ?film dbp:name ?name .
   FILTER(LANG(?abstract) = 'en')
}
LIMIT 5";

Console.WriteLine("=== Films d'aventure enrichis ===");
ExecuteAndDisplaySparqlQuery(endpoint, q7, 5);

=== Films d'aventure enrichis ===


Aucun resultat trouve.


In [ ]:
// Requete 8 : Comedies romantiques - GROUP_CONCAT, OPTIONAL, FILTER
string q8 = @"
SELECT DISTINCT ?film ?abstract 
       (GROUP_CONCAT(DISTINCT ?starring; SEPARATOR=', ') AS ?acteurs)
       ?director
WHERE {
  ?film dbo:wikiPageWikiLink dbr:Romantic_comedy .
  ?film dbp:starring ?starring .
  ?film rdfs:comment ?abstract .
  OPTIONAL { ?film dbo:director ?director } .
  FILTER(LANG(?abstract) = 'en')
}
LIMIT 5";

Console.WriteLine("=== Comedies romantiques (GROUP_CONCAT) ===");
ExecuteAndDisplaySparqlQuery(endpoint, q8, 5);

=== Comedies romantiques (GROUP_CONCAT) ===


Aucun resultat trouve.


#### Interpretation : Requetes avancees

| Concept | Syntaxe | Effet |
|---------|---------|-------|
| `a dbo:Scientist` | Raccourci pour `rdf:type` | Filtrer par type |
| `ORDER BY ?birth` | Tri par variable | Resultats ordonnes |
| `FILTER(LANG(?x) = 'en')` | Filtre linguistique | Evite les doublons multilingues |
| `OPTIONAL { ... }` | Pattern optionnel | Resultats meme sans match |
| `GROUP_CONCAT(DISTINCT ?x; SEPARATOR=', ')` | Agregation | Une ligne par film, acteurs concatenes |

> **Performance** : Les requetes avec `GROUP_CONCAT` et `OPTIONAL` sont plus lentes. Toujours utiliser `LIMIT` sur les endpoints publics.

---

## 3. Wikidata : L'alternative structuree

**Wikidata** est une base de connaissances libre, collaborative et multilingue (Wikimedia Foundation). Contrairement a DBpedia, elle est **editee manuellement** par une communaute.

| Aspect | DBpedia | Wikidata |
|--------|---------|----------|
| **Source** | Extraction automatique Wikipedia | Saisie manuelle collaborative |
| **Identifiants** | URIs lisibles (`dbr:Paris`) | QIDs (`wd:Q90`) |
| **Proprietes** | `dbo:`, `dbp:` | PIDs (`P31` = instance de) |
| **Endpoint** | `http://dbpedia.org/sparql` | `https://query.wikidata.org/sparql` |
| **Mise a jour** | Periodique (dumps) | Temps reel |
| **Licence** | CC-BY-SA 3.0 | CC0 (domaine public) |

### Conventions Wikidata

| Type | Prefixe | Exemple | Signification |
|------|---------|---------|---------------|
| Entite | `wd:` | `wd:Q937` | Albert Einstein |
| Propriete | `wdt:` | `wdt:P31` | instance de |
| | | `wdt:P161` | distribution (cast member) |
| | | `wdt:P17` | pays |
| Labels | `SERVICE wikibase:label` | `bd:serviceParam wikibase:language 'fr,en'` | Labels lisibles |

In [ ]:
// Connexion a l'endpoint Wikidata
SparqlRemoteEndpoint wikidataEndpoint = new SparqlRemoteEndpoint(
    new Uri("https://query.wikidata.org/sparql")
);

// Requete Wikidata 1 : Proprietes d'Albert Einstein (Q937)
string wd1 = @"
SELECT ?property ?propertyLabel ?value ?valueLabel
WHERE {
  wd:Q937 ?prop ?value .
  ?property wikibase:directClaim ?prop .
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr,en'. }
}
LIMIT 15";

try
{
    Console.WriteLine("=== Albert Einstein sur Wikidata ===");
    ExecuteAndDisplaySparqlQuery(wikidataEndpoint, wd1, 15);
}
catch (Exception ex)
{
    Console.WriteLine($"Erreur Wikidata : {ex.Message}");
    Console.WriteLine("Note : Wikidata peut refuser les requetes sans User-Agent (HTTP 403).");
}

=== Albert Einstein sur Wikidata ===


Erreur SPARQL : A HTTP Error occurred while trying to make the SPARQL Query, see inner exception for details



(2,1): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'

(2,45): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'



In [ ]:
// Requete Wikidata 2 : Films avec Brad Pitt (Q35332)
// P161 = cast member, P31 = instance of, Q11424 = film
string wd2 = @"
SELECT ?film ?filmLabel
WHERE {
  ?film wdt:P161 wd:Q35332 .
  ?film wdt:P31 wd:Q11424 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr,en'. }
}
LIMIT 20";

try
{
    Console.WriteLine("=== Films avec Brad Pitt (Wikidata) ===");
    ExecuteAndDisplaySparqlQuery(wikidataEndpoint, wd2, 20);
}
catch (Exception ex)
{
    Console.WriteLine($"Erreur Wikidata : {ex.Message}");
}

=== Films avec Brad Pitt (Wikidata) ===


Erreur SPARQL : A HTTP Error occurred while trying to make the SPARQL Query, see inner exception for details


In [ ]:
// Requete Wikidata 3 : Grandes villes de France avec population
// Q142 = France, P17 = country, P1082 = population, Q515 = city
string wd3 = @"
SELECT ?city ?cityLabel ?population
WHERE {
  ?city wdt:P31 wd:Q515 .
  ?city wdt:P17 wd:Q142 .
  ?city wdt:P1082 ?population .
  FILTER(?population > 100000)
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr'. }
}
ORDER BY DESC(?population)
LIMIT 20";

try
{
    Console.WriteLine("=== Grandes villes de France (Wikidata) ===");
    ExecuteAndDisplaySparqlQuery(wikidataEndpoint, wd3, 20);
}
catch (Exception ex)
{
    Console.WriteLine($"Erreur Wikidata : {ex.Message}");
}

=== Grandes villes de France (Wikidata) ===


Erreur SPARQL : A HTTP Error occurred while trying to make the SPARQL Query, see inner exception for details


#### Interpretation : DBpedia vs Wikidata

| Aspect | DBpedia | Wikidata |
|--------|---------|----------|
| Einstein | `dbr:Albert_Einstein` | `wd:Q937` |
| Brad Pitt acteur | `dbo:starring` | `wdt:P161` (cast member) |
| Filtre type film | Implicite | `wdt:P31 wd:Q11424` |
| Labels | URIs lisibles | `SERVICE wikibase:label` |
| Population | Non disponible facilement | `wdt:P1082` |

> **Avantage Wikidata** : Donnees plus structurees, a jour, et filtrage precis par type (`P31`). Au prix d'identifiants moins lisibles (QIDs). Cherchez les QIDs sur [wikidata.org](https://www.wikidata.org/).

---

## 4. Requetes federees avec SERVICE

La clause **SERVICE** de SPARQL 1.1 permet d'interroger **plusieurs endpoints** dans une seule requete. C'est le coeur du Linked Data.

```sparql
SELECT ?x ?y ?z
WHERE {
    ?x localPredicate ?y .           # Endpoint principal
    SERVICE <http://autre/sparql> {   # Endpoint distant
        ?y distantPredicate ?z .
    }
}
```

En pratique, les vrais appels `SERVICE` sont souvent bloques par les endpoints. L'approche pragmatique utilise `owl:sameAs` pour relier les entites entre datasets.

> **Attention** : Les requetes federees sont souvent lentes et peuvent etre rejetees. Toujours prevoir un fallback.

In [ ]:
// Requete federee via owl:sameAs : relier DBpedia et Wikidata
// owl:sameAs lie des entites equivalentes entre datasets
string fedQuery = @"
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?person ?name ?wikidataId
WHERE {
  ?person a dbo:Scientist .
  ?person dbo:award dbr:Nobel_Prize_in_Physics .
  ?person rdfs:label ?name .
  ?person owl:sameAs ?wikidataId .
  FILTER(LANG(?name) = 'en')
  FILTER(STRSTARTS(STR(?wikidataId), 'http://www.wikidata.org/'))
}
LIMIT 10";

try
{
    Console.WriteLine("=== Laureats Nobel avec liens Wikidata (owl:sameAs) ===");
    ExecuteAndDisplaySparqlQuery(endpoint, fedQuery);
}
catch (Exception ex)
{
    Console.WriteLine($"Erreur : {ex.Message}");
}

=== Laureats Nobel avec liens Wikidata (owl:sameAs) ===


?person = http://dbpedia.org/resource/Arthur_Leonard_Schawlow , ?name = Arthur Leonard Schawlow@en , ?wikidataId = http://www.wikidata.org/entity/Q190503


?person = http://dbpedia.org/resource/Arthur_Leonard_Schawlow , ?name = Arthur Leonard Schawlow@en , ?wikidataId = http://www.wikidata.org/entity/Q190503


?person = http://dbpedia.org/resource/Arthur_Leonard_Schawlow , ?name = Arthur Leonard Schawlow@en , ?wikidataId = http://www.wikidata.org/entity/Q190503


?person = http://dbpedia.org/resource/Claude_Cohen-Tannoudji , ?name = Claude Cohen-Tannoudji@en , ?wikidataId = http://www.wikidata.org/entity/Q190697


...

?person = http://dbpedia.org/resource/Ernst_Ruska , ?name = Ernst Ruska@en , ?wikidataId = http://www.wikidata.org/entity/Q71022


?person = http://dbpedia.org/resource/Ernst_Ruska , ?name = Ernst Ruska@en , ?wikidataId = http://www.wikidata.org/entity/Q71022


?person = http://dbpedia.org/resource/Ernst_Ruska , ?name = Ernst Ruska@en , ?wikidataId = http://www.wikidata.org/entity/Q71022


?person = http://dbpedia.org/resource/Georg_Bednorz , ?name = Georg Bednorz@en , ?wikidataId = http://www.wikidata.org/entity/Q76687


...affiche 10 resultats sur 10.


In [ ]:
// Approche en 2 etapes : DBpedia -> owl:sameAs -> Wikidata

// Etape 1 : Obtenir le lien Wikidata depuis DBpedia
string step1 = @"
SELECT ?sameAs
WHERE {
  <http://dbpedia.org/resource/Albert_Einstein> owl:sameAs ?sameAs .
  FILTER(STRSTARTS(STR(?sameAs), 'http://www.wikidata.org/'))
}";

try
{
    Console.WriteLine("=== Etape 1 : owl:sameAs DBpedia -> Wikidata ===");
    ExecuteAndDisplaySparqlQuery(endpoint, step1);

    // Etape 2 : Interroger Wikidata avec le QID obtenu
    string step2 = @"
    SELECT ?propertyLabel ?valueLabel
    WHERE {
      wd:Q937 ?prop ?value .
      ?property wikibase:directClaim ?prop .
      FILTER(?prop IN (wdt:P19, wdt:P69, wdt:P166, wdt:P106))
      SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr,en'. }
    }
    LIMIT 10";

    Console.WriteLine("\n=== Etape 2 : Details depuis Wikidata (Q937) ===");
    ExecuteAndDisplaySparqlQuery(wikidataEndpoint, step2);
}
catch (Exception ex)
{
    Console.WriteLine($"Erreur : {ex.Message}");
}

=== Etape 1 : owl:sameAs DBpedia -> Wikidata ===


?sameAs = http://www.wikidata.org/entity/Q937



=== Etape 2 : Details depuis Wikidata (Q937) ===


Erreur SPARQL : A HTTP Error occurred while trying to make the SPARQL Query, see inner exception for details


#### Interpretation : Requetes federees

| Approche | Avantage | Inconvenient |
|----------|----------|-------------|
| `SERVICE` direct | Une seule requete | Souvent bloque, lent |
| `owl:sameAs` + 2 requetes | Fiable, rapide | Code plus complexe |

**Predicat `owl:sameAs`** : Lie des entites equivalentes entre datasets. Exemples :
- `dbr:Albert_Einstein owl:sameAs wd:Q937`
- `dbr:Paris owl:sameAs wd:Q90`

**Proprietes filtrees (etape 2)** : `P19` lieu de naissance, `P69` formation, `P166` distinctions, `P106` occupation.

---

## 5. SparqlRemoteEndpoint en .NET : gestion des timeouts et erreurs

En production, il faut gerer les problemes courants des endpoints distants : timeouts, indisponibilite, limites de requetes. dotNetRDF permet aussi de sauvegarder et recharger des resultats SPARQL dans plusieurs formats.

In [ ]:
// Sauvegarde et rechargement de resultats SPARQL
try
{
    SparqlRemoteEndpoint ep = new SparqlRemoteEndpoint(new Uri("http://dbpedia.org/sparql"));
    SparqlResultSet results = ep.QueryWithResultSet(
        "SELECT DISTINCT ?type WHERE { ?s a ?type } LIMIT 50"
    );

    // Sauvegarder en JSON et XML
    var jsonWriter = new SparqlJsonWriter();
    jsonWriter.Save(results, "data/example.srj");
    Console.WriteLine($"Sauvegarde JSON : {results.Count} resultats -> data/example.srj");

    var xmlWriter = new SparqlXmlWriter();
    xmlWriter.Save(results, "data/example.srx");
    Console.WriteLine($"Sauvegarde XML : {results.Count} resultats -> data/example.srx");

    // Recharger depuis fichier (utile hors-ligne)
    var parser = new SparqlXmlParser();
    SparqlResultSet reloaded = new SparqlResultSet();
    parser.Load(reloaded, "data/example.srx");
    Console.WriteLine($"\nRecharge depuis XML : {reloaded.Count} resultats");
    Console.WriteLine($"Variables : {string.Join(", ", reloaded.Variables)}");
}
catch (Exception ex)
{
    Console.WriteLine($"Erreur endpoint : {ex.Message}");
    
    // Fallback : lire les fichiers locaux pre-calcules
    if (System.IO.File.Exists("data/example.srx"))
    {
        var parser = new SparqlXmlParser();
        SparqlResultSet local = new SparqlResultSet();
        parser.Load(local, "data/example.srx");
        Console.WriteLine($"Fallback local : {local.Count} resultats");
    }
    else
    {
        Console.WriteLine("Aucun fichier local disponible.");
    }
}

Sauvegarde JSON : 50 resultats -> data/example.srj


Sauvegarde XML : 50 resultats -> data/example.srx



Recharge depuis XML : 50 resultats


Variables : type



(4,5): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'

(4,35): warning CS0618: 'SparqlRemoteEndpoint' est obsolète : 'This class is obsolete and will be removed in a future release. Replaced by VDS.RDF.Query.SparqlQueryClient.'



#### Interpretation : Formats et gestion d'erreurs

| Format | Extension | Writer | Parser | Usage |
|--------|-----------|--------|--------|-------|
| XML | `.srx` | `SparqlXmlWriter` | `SparqlXmlParser` | Standard W3C |
| JSON | `.srj` | `SparqlJsonWriter` | `SparqlJsonParser` | APIs web |
| CSV | `.csv` | `SparqlCsvWriter` | `SparqlCsvParser` | Tableurs |

**Bonnes pratiques** :
1. Toujours entourer les appels endpoint de `try-catch`
2. Prevoir un fallback sur fichiers locaux pour le mode hors-ligne
3. Utiliser `LIMIT` sur les endpoints publics (quotas de 10-60 secondes)
4. Sauvegarder les resultats importants localement

---

## Exercices pratiques

### Exercice 1 : Interroger DBpedia pour une personne de votre choix

Choisissez une personne celebre et ecrivez une requete SPARQL pour obtenir ses informations depuis DBpedia.

**Indices** :
- URI DBpedia : `http://dbpedia.org/resource/Prenom_Nom`
- Testez d'abord sur [DBpedia SPARQL](http://dbpedia.org/sparql)

In [ ]:
// Exercice 1 : Remplacez l'URI par une personne de votre choix
// Exemple : http://dbpedia.org/resource/Marie_Curie

// string exQuery = "SELECT ?property ?value WHERE { <http://dbpedia.org/resource/Marie_Curie> ?property ?value . } LIMIT 20";
// ExecuteAndDisplaySparqlQuery(endpoint, exQuery, 20);

### Exercice 2 : Meme personne sur Wikidata

Trouvez la meme personne sur Wikidata et comparez les resultats avec DBpedia.

**Indices** :
- Cherchez le QID sur [wikidata.org](https://www.wikidata.org/) (ex : Marie Curie = Q7186)
- Utilisez `SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr,en'. }`

In [ ]:
// Exercice 2 : Remplacez Q7186 par le QID de votre personne

// string exWd = @"
// SELECT ?propertyLabel ?valueLabel
// WHERE {
//   wd:Q7186 ?prop ?value .
//   ?property wikibase:directClaim ?prop .
//   SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr,en'. }
// }
// LIMIT 20";
// ExecuteAndDisplaySparqlQuery(wikidataEndpoint, exWd, 20);

### Exercice 3 : Pont owl:sameAs entre DBpedia et Wikidata

Ecrivez deux requetes qui exploitent `owl:sameAs` pour relier DBpedia et Wikidata sur un sujet de votre choix.

**Indices** :
- Etape 1 : `SELECT ?sameAs WHERE { <dbr:SUJET> owl:sameAs ?sameAs . FILTER(STRSTARTS(STR(?sameAs), 'http://www.wikidata.org/')) }`
- Etape 2 : Utilisez le QID obtenu pour interroger Wikidata

In [ ]:
// Exercice 3 : Trouvez les liens owl:sameAs puis interrogez Wikidata

// Etape 1 : Liens owl:sameAs depuis DBpedia
// string sameAsQ = @"
// SELECT ?sameAs
// WHERE {
//   <http://dbpedia.org/resource/Paris> owl:sameAs ?sameAs .
//   FILTER(STRSTARTS(STR(?sameAs), 'http://www.wikidata.org/'))
// }";
// ExecuteAndDisplaySparqlQuery(endpoint, sameAsQ);

// Etape 2 : Details Wikidata (Q90 = Paris)
// string wdQ = @"
// SELECT ?propertyLabel ?valueLabel
// WHERE {
//   wd:Q90 ?prop ?value .
//   ?property wikibase:directClaim ?prop .
//   SERVICE wikibase:label { bd:serviceParam wikibase:language 'fr'. }
// }
// LIMIT 15";
// ExecuteAndDisplaySparqlQuery(wikidataEndpoint, wdQ, 15);

---

## Resume

| Concept | Description |
|---------|-------------|
| **4 regles du Linked Data** | URIs, HTTP, informations utiles, liens entre sources |
| **5 etoiles Open Data** | Graduation de la qualite des donnees ouvertes |
| **DBpedia** | Extraction Wikipedia, URIs lisibles, endpoint `http://dbpedia.org/sparql` |
| **Wikidata** | Base collaborative, QIDs/PIDs, `SERVICE wikibase:label` |
| **owl:sameAs** | Pont entre datasets equivalents (DBpedia <-> Wikidata) |
| **Requetes federees** | Clause `SERVICE` pour combiner endpoints |
| **SparqlRemoteEndpoint** | Classe .NET pour interroger des endpoints distants |
| **Gestion d'erreurs** | `try-catch`, fallback local, `LIMIT` obligatoire |

### Quand utiliser quel endpoint ?

| Scenario | Recommandation | Raison |
|----------|---------------|--------|
| Prototype rapide | DBpedia | URIs lisibles |
| Production | Wikidata | Qualite, fraicheur, CC0 |
| Texte Wikipedia | DBpedia | Abstracts disponibles |
| Combinaison | Les deux + `owl:sameAs` | Complementarite |

**Pour aller plus loin** :
- [LOD Cloud](https://lod-cloud.net/) : GeoNames, MusicBrainz, UniProt
- [SPARQL Federation](https://www.w3.org/TR/sparql11-federated-query/) (W3C)
- [Wikidata Query Service](https://query.wikidata.org/) (editeur visuel)

**Prochain notebook** : [SW-6-RDFS](SW-6-RDFS.ipynb) - Vocabulaire RDFS, inference et hierarchies de classes

---

**Navigation** : [<< 4-SPARQL](SW-4-SPARQL.ipynb) | [Index](README.md) | [6-RDFS >>](SW-6-RDFS.ipynb)